Data processing:

In [1]:
import pandas as pd
import numpy as np

Modeling:

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

Tree Visualisation:

In [4]:
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz